In [ ]:
#!git clone https://github.com/zhouhaoyi/Informer2020.git
#!git clone https://github.com/zhouhaoyi/ETDataset.git
# modify the code based on my dataset

In [ ]:
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install scikit_learn
# !{sys.executable} -m pip install torch
# !{sys.executable} -m pip install seaborn

In [1]:
import sys
import os
import pandas as pd
import numpy as np
from argparse import Namespace
import re
from sklearn.metrics import (
    mean_absolute_error,
    root_mean_squared_error,
    mean_absolute_percentage_error
)

In [2]:
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

from data.data_loader import Dataset_Custom
from utils.metrics import metric

In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.8.0+cu128
12.8
True


In [4]:
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

for i in range(num_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    print(f"GPU {i}: {gpu_name}")

Number of GPUs available: 1
GPU 0: NVIDIA A30


## Prepare the data

In [5]:
my_data = pd.read_csv('../GD030A_S.csv')

In [6]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

Custom data (xxx.csv) has to include at least 2 features: date(format: YYYY-MM-DD hh:mm:ss) and target feature.

In [ ]:
traffic_full = recover_timestamp(my_data)
traffic_full.drop(['date', 'time'], axis=1, inplace=True)
traffic_full = traffic_full.reset_index(names='date')
traffic_full['date'] = traffic_full['date'].astype(str)
traffic_full

In [ ]:
#traffic_full.to_csv('traffic_full.csv', index=False)

In [60]:
# after covid data
traffic_full = pd.read_csv('../traffic_full.csv')
traffic_full
traffic_after_covid = traffic_full.loc[traffic_full[traffic_full['date'] == '2022-06-03 00:00:00'].index[0]:]
traffic_after_covid = traffic_after_covid.reset_index(drop=True)

In [59]:
#traffic_after_covid.to_csv('../traffic_after_covid.csv', index=False)

## Run training directly using bash

In [5]:
# ensure the shell commands below using the current Python environment
os.environ["PATH"] = f"{sys.prefix}/bin:" + os.environ["PATH"]

In [6]:
cd Informer2020

/home/yl2672496l/Yue/code/transformer/Informer2020


In [7]:
#test
# !python -u main_informer.py --model informer --data traffic_after_covid --root_path ../ --features S --seq_len 24 --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 1 #--inverse

In [8]:
seq_lens = [24 * i for i in range(1, 22)]
print(seq_lens)

[24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360, 384, 408, 432, 456, 480, 504]


## rerun the code with updated label_len

In [ ]:
# full dataset
# for seq_len in seq_lens:
#     print(f"Running seq_len={seq_len}")
#     !python -u main_informer.py --model informer --data traffic_full --root_path ../ --features S --seq_len {seq_len} --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10

for seq_len in seq_lens:
    # Calculate label_len: max(pred_len, seq_len // 3)
    label_len = max(6, seq_len // 3)
    
    print(f"Running seq_len={seq_len}, label_len={label_len}, pred_len=6")
    !python -u main_informer.py --model informer --data traffic_full --root_path ../ --features S --seq_len {seq_len} --label_len {label_len} --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10

Running seq_len=24
Args in experiment:
Namespace(model='informer', data='traffic_full', root_path='../', data_path='traffic_full.csv', features='S', target='flow', freq='h', checkpoints='./checkpoints/', seq_len=24, label_len=24, pred_len=6, enc_in=1, dec_in=1, c_out=1, d_model=512, n_heads=8, e_layers=2, d_layers=1, s_layers=[3, 2, 1], d_ff=2048, factor=5, padding=0, distil=True, dropout=0.05, attn='prob', embed='timeF', activation='gelu', output_attention=False, do_predict=False, mix=True, cols=None, num_workers=0, itr=10, train_epochs=6, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', loss='mse', lradj='type1', use_amp=False, inverse=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', detail_freq='h')
Use GPU: cuda:0
>>>>>>>start training : informer_traffic_full_ftS_sl24_ll24_pl6_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 24515
[DEBUG] Original dataset length: 24515
[DEBUG] Cleaned dataset length (NaN r

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



	iters: 200, epoch: 2 | loss: 0.1532303
	speed: 0.0437s/iter; left time: 126.2771s
	iters: 300, epoch: 2 | loss: 0.1194427
	speed: 0.0421s/iter; left time: 117.3745s
	iters: 400, epoch: 2 | loss: 0.1741949
	speed: 0.0420s/iter; left time: 113.1091s
	iters: 500, epoch: 2 | loss: 0.2344944
	speed: 0.0424s/iter; left time: 109.8542s
	iters: 600, epoch: 2 | loss: 0.2087820
	speed: 0.0433s/iter; left time: 107.9172s
Epoch: 2 cost time: 26.315322875976562
Epoch: 2, Steps: 618 | Train Loss: 0.2002093 Vali Loss: 0.3317700 Test Loss: 0.2666194
Validation loss decreased (0.349521 --> 0.331770).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1226939
	speed: 0.0955s/iter; left time: 226.5762s
	iters: 200, epoch: 3 | loss: 0.1673248
	speed: 0.0435s/iter; left time: 98.8637s
	iters: 300, epoch: 3 | loss: 0.1549045
	speed: 0.0422s/iter; left time: 91.8031s
	iters: 400, epoch: 3 | loss: 0.1939242
	speed: 0.0417s/iter; left time: 86.5266s
	iters: 500, epoch: 3 | loss:

In [ ]:
# after covid dataset
for seq_len in seq_lens:
    print(f"Running seq_len={seq_len}")
    !python -u main_informer.py --model informer --data traffic_after_covid --root_path ../ --features S --seq_len {seq_len} --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10

Running seq_len=24
Args in experiment:
Namespace(model='informer', data='traffic_after_covid', root_path='../', data_path='traffic_after_covid.csv', features='S', target='flow', freq='h', checkpoints='./checkpoints/', seq_len=24, label_len=24, pred_len=6, enc_in=1, dec_in=1, c_out=1, d_model=512, n_heads=8, e_layers=2, d_layers=1, s_layers=[3, 2, 1], d_ff=2048, factor=5, padding=0, distil=True, dropout=0.05, attn='prob', embed='timeF', activation='gelu', output_attention=False, do_predict=False, mix=True, cols=None, num_workers=0, itr=10, train_epochs=6, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', loss='mse', lradj='type1', use_amp=False, inverse=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', detail_freq='h')
Use GPU: cuda:0
>>>>>>>start training : informer_traffic_after_covid_ftS_sl24_ll24_pl6_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 6955
[DEBUG] Original dataset length: 6955
[DEBUG] Cleaned da

In [10]:
data = np.load("./results/running_time_24.npy")
data

array([5.38114933, 0.54586172])

## Calculate the metrics

In [28]:
# Extract seq_len and run number from directory name
# Example:
# informer_traffic_full_ftS_sl24_ll24_pl6_dm512_..._Exp_3
# seq_len = 24, run_num = 3
def extract_info(dirname):
    seq_match = re.search(r"sl(\d+)", dirname)
    run_match = re.search(r"Exp_(\d+)", dirname)

    seq_len = int(seq_match.group(1)) if seq_match else None
    run_num = int(run_match.group(1)) + 1 if run_match else None  # Exp_0 → run 1

    return seq_len, run_num

# Compute metrics for each of 6 prediction steps
# pred, true shape: (N, 6, 1)
def compute_step_metrics(pred, true):
    steps = pred.shape[1]
    rows = []

    for step in range(steps):
        p = pred[:, step, 0]
        g = true[:, step, 0]

        mae = mean_absolute_error(g, p)
        rmse = root_mean_squared_error(g, p)
        mape = mean_absolute_percentage_error(g, p) * 100  # convert to percentage
        rows.append((step + 1, mae, rmse, mape))
        
    return rows

Metrics of each run and each step (6×10×21)

In [50]:
def whole_metrics(target_dataset):
    all_rows = []
    
    # your result directories (you can replace this with auto-search)
    result_dirs = sorted([
        d for d in os.listdir("results")
        if os.path.isdir(os.path.join("results", d)) and f"informer_{target_dataset}_" in d  # <<< key filtering step
    ])
    
    for d in result_dirs:
        seq_len, run_num = extract_info(d)
    
        pred_path = os.path.join("results", d, "pred_inverse.npy")
        true_path = os.path.join("results", d, "true_inverse.npy")
    
        if not (os.path.exists(pred_path) and os.path.exists(true_path)):
            print(f"Skipping missing: {d}")
            continue
    
        pred = np.load(pred_path)
        true = np.load(true_path)
    
        metrics = compute_step_metrics(pred, true)
    
        for step, mae, rmse, mape in metrics:
            all_rows.append({
                "input_len": seq_len,
                "run_num": run_num,
                "pre_step": step,
                "MAE_test": mae,
                "RMSE_test": rmse,
                "MAPE (%)_test": mape
            })
    
    df = pd.DataFrame(all_rows)
    df = df.sort_values(by=["input_len", "run_num", "pre_step"]).reset_index(drop=True)

    return df

Skipping missing: .ipynb_checkpoints


In [ ]:
metrics_full = whole_metrics("traffic_full")
#metrics_full.to_csv('../all_metrics_informer_entire_data.csv', index=False)

In [ ]:
whole_metrics("traffic_after_covid")

Metrics of each input length (21)

In [51]:
avg_by_seq = metrics_full.groupby("input_len")[["MAE_test", "RMSE_test", "MAPE (%)_test"]].mean().reset_index()
#avg_by_seq.to_csv('../metrics_informer_entire_data.csv', index=False)
avg_by_seq

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.827424,40.091506,33.976340
1,48,27.086889,40.371117,35.322956
2,72,27.484757,40.414853,37.142969
3,96,27.565523,41.038521,35.855909
4,120,27.364081,41.213944,35.366292
5,144,27.528013,41.292334,35.943623
6,168,27.900328,41.459094,37.013240
7,192,27.311530,41.252090,34.981317
8,216,27.348362,40.972219,37.868599
9,240,27.662873,41.741723,37.931720


In [46]:
# Collect all result directories
target_dataset = "traffic_full"
result_dirs = sorted([
    d for d in os.listdir("results")
    if os.path.isdir(os.path.join("results", d)) and f"informer_{target_dataset}_" in d  # <<< key filtering step
])

# Dictionary to store metrics per seq_len
metrics_dict = {}

for d in result_dirs:
    folder_path = os.path.join("results", d)
    
    metrics_path = os.path.join(folder_path, "metrics_inverse.npy")
    if not os.path.exists(metrics_path):
        print(f"[WARNING] Missing metrics_inverse.npy in {folder_path}")
        continue
    
    metrics = np.load(metrics_path)  # [MAE, MSE, RMSE, MAPE, MSPE]
    
    # Extract seq_len from folder name (assuming folder name contains 'sl<seq_len>_run<run_num>')
    seq_len = int([s for s in d.split("_") if s.startswith("sl")][0][2:])
    
    if seq_len not in metrics_dict:
        metrics_dict[seq_len] = {"MAE": [], "RMSE": [], "MAPE": []}
    
    metrics_dict[seq_len]["MAE"].append(metrics[0])
    metrics_dict[seq_len]["RMSE"].append(metrics[2])
    metrics_dict[seq_len]["MAPE"].append(metrics[3])

# Calculate mean over all runs per seq_len
rows = []
for seq_len, vals in metrics_dict.items():
    rows.append({
        "input_len": seq_len,
        "MAE_test": np.mean(vals["MAE"]),
        "RMSE_test": np.mean(vals["RMSE"]),
        "MAPE (%)_test": np.mean(vals["MAPE"]) *100
    })

df_inverse_avg = pd.DataFrame(rows).sort_values("input_len").reset_index(drop=True)
df_inverse_avg

[WARNING] Missing metrics_inverse.npy in results/.ipynb_checkpoints


,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.827423,40.248596,33.976345
1,48,27.086889,40.532936,35.322960
2,72,27.484756,40.567635,37.142967
3,96,27.565521,41.195618,35.855907
4,120,27.364080,41.375389,35.366291
5,144,27.528015,41.451664,35.943626
6,168,27.900330,41.608902,37.013237
7,192,27.311533,41.407749,34.981316
8,216,27.348362,41.117546,37.868603
9,240,27.662872,41.893860,37.931721


In [48]:
avg_by_seq.equals(df_inverse_avg)

False

In [49]:
avg_by_seq.compare(df_inverse_avg)  

MAE_test             RMSE_test            MAPE (%)_test           
         self      other       self      other          self      other
0   26.827424  26.827423  40.091506  40.248596     33.976340  33.976345
1   27.086889  27.086889  40.371117  40.532936     35.322956  35.322960
2   27.484757  27.484756  40.414853  40.567635     37.142969  37.142967
3   27.565523  27.565521  41.038521  41.195618     35.855909  35.855907
4   27.364081  27.364080  41.213944  41.375389     35.366292  35.366291
5   27.528013  27.528015  41.292334  41.451664     35.943623  35.943626
6   27.900328  27.900330  41.459094  41.608902     37.013240  37.013237
7   27.311530  27.311533  41.252090  41.407749     34.981317  34.981316
8   27.348362  27.348362  40.972219  41.117546     37.868599  37.868603
9   27.662873  27.662872  41.741723  41.893860     37.931720  37.931721
10  27.909886  27.909887  42.613484  42.781487     36.033699  36.033703
11  28.428512  28.428513  42.839811  43.016197     38.538594  38.538597
12  27.657373  27.657373  42.875749  43.059753     34.077934  34.077934
13  28.042166  28.042166  42.898795  43.075539     35.931267  35.931267
14  28.378985  28.378986  43.788344  43.971962     34.976826  34.976826
15  28.860072  28.860071  44.196937  44.398430     36.085584  36.085587
16  29.124343  29.124344  44.124789  44.320198     38.010948  38.010944
17  28.937098  28.937099  44.651621  44.846455     36.279675  36.279671
18  29.117829  29.117828  44.748922  44.940174     37.068310  37.068310
19  29.624438  29.624439  46.003536  46.211472     36.347774  36.347771
20  29.368494  29.368494  46.124282  46.341087     36.350152  36.350151